# 🎯 DeepEval: Evaluating LangChain Agents with LLM-as-Judge

[![DeepEval](https://img.shields.io/badge/DeepEval-Latest-purple.svg)](https://docs.confident-ai.com/)
[![LangChain](https://img.shields.io/badge/LangChain-0.2+-green.svg)](https://python.langchain.com/)
[![Python](https://img.shields.io/badge/Python-3.11+-blue.svg)](https://python.org)

---

## 📚 What You'll Learn

This notebook demonstrates how to evaluate AI agents using **DeepEval's LLM-as-Judge** approach:

| Section | Topic |
|---------|-------|
| 🔧 Setup | Environment and API configuration |
| 🛠️ Tools | LangChain tools for loan calculations |
| 🤖 Agent | Building a ReAct agent with LangGraph |
| 📝 Goldens | Test templates covering multiple scenarios |
| 📊 Metrics | **Reference-based** vs **Referenceless** evaluation |
| 🔬 Evaluation | Running DeepEval on agent outputs |
| 📈 Analysis | Understanding and interpreting results |

---

## 🎓 Key Concept: Reference vs Referenceless Metrics

DeepEval metrics fall into two categories:

### 📋 Reference-Based Metrics
> **Require ground truth** (`expected_output` or `retrieval_context`)

| Metric | Required Parameter | What It Measures |
|--------|-------------------|------------------|
| `Faithfulness` | `retrieval_context` | Is output grounded in provided context? |
| `Hallucination` | `context` | Does output contain fabricated info? |
| `ContextualRecall` | `expected_output` + `retrieval_context` | Does context contain all needed info? |
| `ContextualPrecision` | `expected_output` + `retrieval_context` | Is context focused and relevant? |

### 🆓 Referenceless Metrics  
> **No ground truth needed** - evaluate output quality directly

| Metric | Required Parameter | What It Measures |
|--------|-------------------|------------------|
| `AnswerRelevancy` | `input` only | Is response relevant to the question? |
| `Toxicity` | `actual_output` only | Contains harmful content? |
| `Bias` | `actual_output` only | Contains biased language? |
| `TaskCompletion` | Agent trace | Did agent complete the task? |

---

## 🏦 Demo: Loan Advisor Agent

We'll evaluate a **multi-type loan advisor** with 6 tools:

| Tool | Loan Type | Description |
|------|-----------|-------------|
| `calculate_personal_loan` | 💳 Personal | Unsecured loan payment calculation |
| `calculate_mortgage` | 🏠 Mortgage | Home loan with LTV & PMI analysis |
| `calculate_auto_loan` | 🚗 Auto | Car loan with trade-in support |
| `check_loan_eligibility` | ✅ All | Credit & income eligibility check |
| `check_affordability` | 💰 All | DTI-based affordability analysis |
| `compare_loan_options` | 📊 All | Compare different loan terms |

---

## 🔧 Part 1: Environment Setup

In [ ]:
# 📦 Install dependencies (uncomment for Kaggle/Colab)
# !pip install -q deepeval langchain langchain-openai langgraph pandas python-dotenv

In [ ]:
import os
import json
from typing import Dict, Any

# =============================================================================
# 🔑 API Key Setup (Kaggle / Colab / Local)
# =============================================================================

def setup_api_key():
    """Load API key from various sources with priority order."""
    
    # 1️⃣ Environment variable (highest priority)
    if os.getenv("OPENAI_API_KEY"):
        return "environment"
    
    # 2️⃣ Kaggle Secrets
    try:
        from kaggle_secrets import UserSecretsClient
        os.environ["OPENAI_API_KEY"] = UserSecretsClient().get_secret("OPENAI_API_KEY")
        return "Kaggle Secrets"
    except: pass
    
    # 3️⃣ Google Colab Secrets
    try:
        from google.colab import userdata
        os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
        return "Colab Secrets"
    except: pass
    
    # 4️⃣ Local .env file
    try:
        from dotenv import load_dotenv
        load_dotenv()
        if os.getenv("OPENAI_API_KEY"):
            return ".env file"
    except: pass
    
    return None

source = setup_api_key()
if source:
    print(f"✅ API key loaded from {source}")
else:
    raise ValueError("❌ OPENAI_API_KEY not found. Set via Kaggle/Colab Secrets or .env file.")

## 🛠️ Part 2: LangChain Tools Definition

<details>
<summary>ℹ️ <b>About This Section</b> (click to expand)</summary>

The next cell contains **6 LangChain tools** for loan calculations:

| Tool | Description |
|------|-------------|
| `calculate_personal_loan` | Personal loan payment calculation |
| `calculate_mortgage` | Home loan with down payment & LTV ratio |
| `calculate_auto_loan` | Car loan with trade-in support |
| `check_loan_eligibility` | Credit & income eligibility check |
| `check_affordability` | DTI-based affordability analysis |
| `compare_loan_options` | Compare different loan terms |

**💡 Tip:** You can **collapse the next code cell** in Kaggle/Jupyter by clicking the blue bar on the left. The tools are standard LangChain `@tool` decorated functions.

</details>

---

In [ ]:
%%writefile langchain_tools.py
# =============================================================================
# 🛠️ LangChain Tools for Loan Advisor Agent
# =============================================================================
# 💡 TIP: Collapse this cell in Kaggle/Jupyter - just run it once to create the file!
# =============================================================================

"""LangChain Tools for Loan Advisor Agent.

Tools:
    - calculate_personal_loan: Personal loan payment calculation
    - calculate_mortgage: Mortgage/home loan with down payment and LTV
    - calculate_auto_loan: Auto/car loan calculation
    - check_loan_eligibility: Check if applicant qualifies for a loan
    - check_affordability: Analyze debt-to-income ratio
    - compare_loan_options: Compare different loan terms
"""

import json
from langchain_core.tools import tool

# =============================================================================
# Core Calculation Functions
# =============================================================================

def _calculate_monthly_payment(principal: float, annual_rate: float, months: int) -> float:
    """Calculate monthly payment using amortization formula."""
    if annual_rate == 0:
        return principal / months
    monthly_rate = annual_rate / 12
    payment = principal * (
        monthly_rate * (1 + monthly_rate) ** months
    ) / ((1 + monthly_rate) ** months - 1)
    return round(payment, 2)

def _calculate_totals(principal: float, monthly_payment: float, months: int) -> dict:
    """Calculate total payment and interest."""
    total_payment = monthly_payment * months
    total_interest = total_payment - principal
    return {
        "total_payment": round(total_payment, 2),
        "total_interest": round(total_interest, 2),
        "interest_percentage": round(total_interest / principal * 100, 2),
    }

# =============================================================================
# 💳 Personal Loan Tool
# =============================================================================

@tool
def calculate_personal_loan(
    loan_amount: float,
    annual_interest_rate: float,
    loan_term_months: int,
) -> str:
    """Calculate monthly payment for a personal loan.

    Args:
        loan_amount: Principal loan amount in dollars (e.g., 25000)
        annual_interest_rate: Annual interest rate as decimal (e.g., 0.10 for 10%)
        loan_term_months: Loan term in months (typically 12-84)

    Returns:
        JSON with monthly_payment, total_payment, total_interest, and loan details
    """
    monthly = _calculate_monthly_payment(loan_amount, annual_interest_rate, loan_term_months)
    totals = _calculate_totals(loan_amount, monthly, loan_term_months)
    result = {
        "loan_type": "personal",
        "loan_amount": loan_amount,
        "annual_interest_rate": annual_interest_rate,
        "loan_term_months": loan_term_months,
        "monthly_payment": monthly,
        **totals,
    }
    return json.dumps(result)

# =============================================================================
# 🏠 Mortgage (Home Loan) Tool
# =============================================================================

@tool
def calculate_mortgage(
    home_price: float,
    down_payment_percent: float,
    annual_interest_rate: float,
    loan_term_years: int,
) -> str:
    """Calculate monthly payment for a mortgage (home loan).

    Args:
        home_price: Total home price in dollars (e.g., 500000)
        down_payment_percent: Down payment as percentage (e.g., 0.20 for 20%)
        annual_interest_rate: Annual interest rate as decimal (e.g., 0.065 for 6.5%)
        loan_term_years: Loan term in years (typically 15 or 30)

    Returns:
        JSON with monthly_payment, loan_amount, down_payment, LTV ratio, and totals
    """
    down_payment = home_price * down_payment_percent
    loan_amount = home_price - down_payment
    loan_term_months = loan_term_years * 12
    ltv_ratio = loan_amount / home_price
    monthly = _calculate_monthly_payment(loan_amount, annual_interest_rate, loan_term_months)
    totals = _calculate_totals(loan_amount, monthly, loan_term_months)
    result = {
        "loan_type": "mortgage",
        "home_price": home_price,
        "down_payment": round(down_payment, 2),
        "down_payment_percent": down_payment_percent,
        "loan_amount": round(loan_amount, 2),
        "ltv_ratio": round(ltv_ratio, 3),
        "annual_interest_rate": annual_interest_rate,
        "loan_term_years": loan_term_years,
        "loan_term_months": loan_term_months,
        "monthly_payment": monthly,
        **totals,
        "ltv_warning": "LTV exceeds 80%, PMI may be required" if ltv_ratio > 0.80 else None,
    }
    return json.dumps(result)

# =============================================================================
# 🚗 Auto Loan (Car Loan) Tool
# =============================================================================

@tool
def calculate_auto_loan(
    vehicle_price: float,
    down_payment: float,
    annual_interest_rate: float,
    loan_term_months: int,
    trade_in_value: float = 0,
) -> str:
    """Calculate monthly payment for an auto (car) loan.

    Args:
        vehicle_price: Total vehicle price in dollars (e.g., 35000)
        down_payment: Down payment amount in dollars (e.g., 5000)
        annual_interest_rate: Annual interest rate as decimal (e.g., 0.059 for 5.9%)
        loan_term_months: Loan term in months (typically 36, 48, 60, or 72)
        trade_in_value: Value of trade-in vehicle, if any (default 0)

    Returns:
        JSON with monthly_payment, loan_amount, effective_price, and totals
    """
    effective_price = vehicle_price - trade_in_value
    loan_amount = effective_price - down_payment
    ltv_ratio = loan_amount / vehicle_price
    monthly = _calculate_monthly_payment(loan_amount, annual_interest_rate, loan_term_months)
    totals = _calculate_totals(loan_amount, monthly, loan_term_months)
    result = {
        "loan_type": "auto",
        "vehicle_price": vehicle_price,
        "trade_in_value": trade_in_value,
        "effective_price": round(effective_price, 2),
        "down_payment": down_payment,
        "loan_amount": round(loan_amount, 2),
        "ltv_ratio": round(ltv_ratio, 3),
        "annual_interest_rate": annual_interest_rate,
        "loan_term_months": loan_term_months,
        "monthly_payment": monthly,
        **totals,
    }
    return json.dumps(result)

# =============================================================================
# ✅ Eligibility Check Tool
# =============================================================================

@tool
def check_loan_eligibility(
    age: int,
    monthly_income: float,
    credit_score: int,
    employment_status: str,
    requested_loan_amount: float,
    loan_type: str = "personal",
) -> str:
    """Check if an applicant is eligible for a loan.

    Args:
        age: Applicant's age in years
        monthly_income: Monthly gross income in dollars
        credit_score: Credit score (300-850 range)
        employment_status: One of: full_time, part_time, self_employed, unemployed
        requested_loan_amount: Desired loan amount in dollars
        loan_type: Type of loan: personal, mortgage, or auto

    Returns:
        JSON with eligibility decision, credit rating, and detailed reasons
    """
    reasons = []
    is_eligible = True

    # Age check
    min_age, max_age = 18, 65
    if age < min_age:
        is_eligible = False
        reasons.append(f"Age {age} below minimum ({min_age})")
    elif age > max_age:
        is_eligible = False
        reasons.append(f"Age {age} above maximum ({max_age})")
    else:
        reasons.append("Age requirement met")

    # Income check
    min_income = 3000 if loan_type == "personal" else 4000
    if monthly_income < min_income:
        is_eligible = False
        reasons.append(f"Income ${monthly_income:,.0f} below minimum (${min_income:,.0f})")
    else:
        reasons.append(f"Income requirement met (${monthly_income:,.0f}/month)")

    # Credit score check
    min_scores = {"personal": 600, "mortgage": 620, "auto": 580}
    min_score = min_scores.get(loan_type, 600)
    if credit_score < min_score:
        is_eligible = False
        credit_rating = "Poor"
        reasons.append(f"Credit score {credit_score} below minimum ({min_score})")
    elif credit_score < 670:
        credit_rating = "Fair"
        reasons.append(f"Credit score acceptable ({credit_score})")
    elif credit_score < 740:
        credit_rating = "Good"
        reasons.append(f"Credit score good ({credit_score})")
    else:
        credit_rating = "Excellent"
        reasons.append(f"Credit score excellent ({credit_score})")

    # Employment check
    if employment_status == "unemployed":
        is_eligible = False
        reasons.append("Employment required for loan approval")
    else:
        reasons.append(f"Employment status acceptable ({employment_status})")

    # DTI estimate
    estimated_payment = requested_loan_amount / 36
    dti_estimate = estimated_payment / monthly_income
    max_dti = 0.43 if loan_type == "mortgage" else 0.50
    if dti_estimate > max_dti:
        is_eligible = False
        reasons.append(f"Estimated DTI {dti_estimate:.1%} exceeds maximum ({max_dti:.0%})")

    result = {
        "loan_type": loan_type,
        "is_eligible": is_eligible,
        "credit_rating": credit_rating,
        "estimated_dti": round(dti_estimate, 3),
        "max_dti": max_dti,
        "reasons": reasons,
        "max_recommended_loan": round(monthly_income * max_dti * 36, 2),
    }
    return json.dumps(result)

# =============================================================================
# 💰 Affordability Check Tool
# =============================================================================

@tool
def check_affordability(
    monthly_income: float,
    existing_monthly_debt: float,
    proposed_loan_amount: float,
    annual_interest_rate: float,
    loan_term_months: int,
) -> str:
    """Analyze if a loan is affordable based on debt-to-income ratio.

    Args:
        monthly_income: Monthly gross income in dollars
        existing_monthly_debt: Current monthly debt payments
        proposed_loan_amount: New loan amount being considered
        annual_interest_rate: Interest rate for the new loan
        loan_term_months: Term of the new loan in months

    Returns:
        JSON with affordability analysis including current/new DTI
    """
    new_payment = _calculate_monthly_payment(
        proposed_loan_amount, annual_interest_rate, loan_term_months
    )
    current_dti = existing_monthly_debt / monthly_income
    new_total_debt = existing_monthly_debt + new_payment
    new_dti = new_total_debt / monthly_income
    max_dti = 0.50
    is_affordable = new_dti <= max_dti
    remaining_capacity = (monthly_income * max_dti) - new_total_debt

    if new_dti <= 0.30:
        assessment = "Excellent - very comfortable debt level"
    elif new_dti <= 0.40:
        assessment = "Good - manageable debt level"
    elif new_dti <= 0.50:
        assessment = "Acceptable - at the higher end of recommended range"
    else:
        assessment = "Not recommended - debt exceeds safe limits"

    result = {
        "is_affordable": is_affordable,
        "monthly_income": monthly_income,
        "existing_monthly_debt": existing_monthly_debt,
        "new_monthly_payment": new_payment,
        "total_monthly_debt": round(new_total_debt, 2),
        "current_dti": round(current_dti, 3),
        "new_dti": round(new_dti, 3),
        "max_dti": max_dti,
        "remaining_capacity": round(max(0, remaining_capacity), 2),
        "assessment": assessment,
    }
    return json.dumps(result)

# =============================================================================
# 📊 Loan Comparison Tool
# =============================================================================

@tool
def compare_loan_options(
    loan_amount: float,
    annual_interest_rate: float,
    term_options: str = "36,48,60",
) -> str:
    """Compare monthly payments across different loan terms.

    Args:
        loan_amount: Principal loan amount in dollars
        annual_interest_rate: Annual interest rate as decimal
        term_options: Comma-separated loan terms in months (default: "36,48,60")

    Returns:
        JSON with comparison of monthly payment and total interest for each term
    """
    terms = [int(t.strip()) for t in term_options.split(",")]
    comparisons = []
    for months in terms:
        monthly = _calculate_monthly_payment(loan_amount, annual_interest_rate, months)
        totals = _calculate_totals(loan_amount, monthly, months)
        comparisons.append({
            "term_months": months,
            "term_years": round(months / 12, 1),
            "monthly_payment": monthly,
            "total_interest": totals["total_interest"],
            "total_payment": totals["total_payment"],
        })
    comparisons.sort(key=lambda x: x["term_months"])
    if len(comparisons) > 1:
        longest = comparisons[-1]
        for comp in comparisons:
            comp["interest_savings_vs_longest"] = round(
                longest["total_interest"] - comp["total_interest"], 2
            )
    result = {
        "loan_amount": loan_amount,
        "annual_interest_rate": annual_interest_rate,
        "comparisons": comparisons,
        "recommendation": (
            f"Shorter terms save money on interest but have higher monthly payments. "
            f"The {comparisons[0]['term_months']}-month option saves "
            f"${comparisons[0].get('interest_savings_vs_longest', 0):,.2f} in interest."
        ) if len(comparisons) > 1 else None,
    }
    return json.dumps(result)

# =============================================================================
# Tool Registry
# =============================================================================

def get_all_tools():
    """Get all available LangChain tools for the loan advisor agent."""
    return [
        calculate_personal_loan,
        calculate_mortgage,
        calculate_auto_loan,
        check_loan_eligibility,
        check_affordability,
        compare_loan_options,
    ]

def get_tool_descriptions() -> str:
    """Get formatted descriptions of all available tools."""
    tools = get_all_tools()
    descriptions = []
    for t in tools:
        descriptions.append(f"- **{t.name}**: {t.description.split('.')[0]}")
    return "\n".join(descriptions)

In [ ]:
# 📥 Import the tools we just created
from langchain_tools import get_all_tools, get_tool_descriptions

tools = get_all_tools()
print(f"✅ Loaded {len(tools)} tools:\n")
print(get_tool_descriptions())

## 🤖 Part 3: Create LangChain ReAct Agent

We use **LangGraph's `create_react_agent`** — a prebuilt ReAct (Reason + Act) agent that:
1. Receives user input
2. Decides which tool(s) to call
3. Executes tools and observes results
4. Generates final response

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# 🧠 Create LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 📋 System prompt defines agent behavior
SYSTEM_PROMPT = """You are a Loan Advisor assistant helping users with:
- Personal loans, mortgages (home loans), and auto (car) loans
- Payment calculations, eligibility checks, and affordability analysis

Use the provided tools for accurate calculations. Be clear and helpful.
Always show the key numbers in your response."""

# 🤖 Create ReAct agent
agent = create_react_agent(llm, tools, prompt=SYSTEM_PROMPT)
print("🤖 Agent created with 6 loan advisor tools")

In [ ]:
# 🏃 Agent Runner - extracts outputs and tool context
class AgentRunner:
    """Wrapper that runs agent and extracts outputs for evaluation."""
    
    def __init__(self, agent):
        self.agent = agent
    
    def run(self, query: str) -> Dict[str, Any]:
        """Run agent and extract actual_output + retrieval_context."""
        result = self.agent.invoke({"messages": [HumanMessage(content=query)]})
        messages = result["messages"]
        
        # 📤 Extract final AI response
        actual_output = next(
            (m.content for m in reversed(messages) if isinstance(m, AIMessage) and m.content),
            ""
        )
        
        # 🔧 Extract tool calls and results
        tools_called = []
        retrieval_context = []  # 💡 Tool outputs become retrieval_context!
        
        for msg in messages:
            if isinstance(msg, AIMessage) and msg.tool_calls:
                tools_called.extend([tc.get("name", "") for tc in msg.tool_calls if isinstance(tc, dict)])
            if isinstance(msg, ToolMessage):
                retrieval_context.append(msg.content)
        
        return {
            "actual_output": actual_output,
            "tools_called": tools_called,
            "retrieval_context": retrieval_context,  # 🎯 Key for Faithfulness/Hallucination metrics!
        }

runner = AgentRunner(agent)
print("✅ AgentRunner ready")

## 📝 Part 4: Define Test Goldens

**Goldens** = test templates that define:
- `input`: User query to test
- `expected_tools`: Which tools should be called
- `expected_keywords`: Key terms that should appear in response

We cover **5 categories** across all loan types:

In [ ]:
GOLDENS = [
    # ==========================================================================
    # 💳 PERSONAL LOAN
    # ==========================================================================
    {
        "id": "personal_loan_basic",
        "category": "💳 Personal",
        "input": "Calculate monthly payment for a $25,000 personal loan at 10% interest for 48 months.",
        "expected_tools": ["calculate_personal_loan"],
        "expected_keywords": ["634", "monthly", "payment"],
    },
    {
        "id": "personal_loan_comparison",
        "category": "💳 Personal",
        "input": "Compare a $20,000 personal loan at 9% interest for 36, 48, and 60 months.",
        "expected_tools": ["compare_loan_options"],
        "expected_keywords": ["36", "48", "60", "interest"],
    },
    
    # ==========================================================================
    # 🏠 MORTGAGE (HOME LOAN)
    # ==========================================================================
    {
        "id": "mortgage_basic",
        "category": "🏠 Mortgage",
        "input": "Calculate mortgage payment for a $500,000 home with 20% down payment at 6.5% for 30 years.",
        "expected_tools": ["calculate_mortgage"],
        "expected_keywords": ["2,528", "monthly", "down payment", "400,000"],
    },
    {
        "id": "mortgage_low_down",
        "category": "🏠 Mortgage",
        "input": "What's the monthly payment for a $400,000 house with only 10% down at 7% for 30 years? Will I need PMI?",
        "expected_tools": ["calculate_mortgage"],
        "expected_keywords": ["LTV", "PMI", "monthly"],
    },
    
    # ==========================================================================
    # 🚗 AUTO LOAN (CAR LOAN)
    # ==========================================================================
    {
        "id": "auto_loan_basic",
        "category": "🚗 Auto",
        "input": "Calculate car loan payment for a $35,000 vehicle with $5,000 down at 5.9% for 60 months.",
        "expected_tools": ["calculate_auto_loan"],
        "expected_keywords": ["581", "monthly", "30,000"],
    },
    {
        "id": "auto_loan_trade_in",
        "category": "🚗 Auto",
        "input": "I want to buy a $40,000 car. I have a trade-in worth $8,000 and can put $2,000 down. What's my payment at 6% for 72 months?",
        "expected_tools": ["calculate_auto_loan"],
        "expected_keywords": ["trade", "monthly", "30,000"],
    },
    
    # ==========================================================================
    # ✅ ELIGIBILITY CHECK
    # ==========================================================================
    {
        "id": "eligibility_good_credit",
        "category": "✅ Eligibility",
        "input": "Check my loan eligibility: age 35, income $8,000/month, credit score 750, full-time employed, requesting $50,000 personal loan.",
        "expected_tools": ["check_loan_eligibility"],
        "expected_keywords": ["eligible", "Excellent", "750"],
    },
    {
        "id": "eligibility_low_credit",
        "category": "✅ Eligibility",
        "input": "Am I eligible for a mortgage? Age 28, income $5,000/month, credit score 580, self-employed, want $300,000.",
        "expected_tools": ["check_loan_eligibility"],
        "expected_keywords": ["not", "credit", "580"],
    },
    
    # ==========================================================================
    # 💰 AFFORDABILITY ANALYSIS
    # ==========================================================================
    {
        "id": "affordability_ok",
        "category": "💰 Affordability",
        "input": "Can I afford a $30,000 car loan at 6% for 60 months? I earn $6,000/month with $500 existing debt.",
        "expected_tools": ["check_affordability"],
        "expected_keywords": ["affordable", "DTI"],
    },
    {
        "id": "affordability_stretched",
        "category": "💰 Affordability",
        "input": "Monthly income $4,000, existing debt $1,500. Can I afford a $25,000 loan at 8% for 48 months?",
        "expected_tools": ["check_affordability"],
        "expected_keywords": ["DTI", "exceed", "not"],
    },
]

# 📊 Summary
print(f"📝 Defined {len(GOLDENS)} test cases:\n")
from collections import Counter
categories = Counter(g["category"] for g in GOLDENS)
for cat, count in categories.items():
    print(f"  {cat}: {count} tests")

## 🏃 Part 5: Run Agent on All Test Cases

Execute the agent for each Golden and collect:
- `actual_output`: Agent's final response
- `retrieval_context`: Tool outputs (used by Faithfulness/Hallucination metrics)

In [ ]:
from deepeval.dataset import EvaluationDataset, Golden

print("🏃 Running agent on all test cases...\n")
print("=" * 60)

goldens_with_output = []

for g in GOLDENS:
    print(f"\n{g['category']} | {g['id']}")
    print(f"  📥 Input: {g['input'][:50]}...")
    
    result = runner.run(g["input"])
    print(f"  🔧 Tools called: {result['tools_called']}")
    
    # Create Golden with agent output
    golden = Golden(
        input=g["input"],
        actual_output=result["actual_output"],
        retrieval_context=result["retrieval_context"],  # 💡 Tool outputs!
        additional_metadata={
            "test_id": g["id"],
            "category": g["category"],
            "expected_tools": g["expected_tools"],
            "actual_tools": result["tools_called"],
            "expected_keywords": g["expected_keywords"],
        }
    )
    goldens_with_output.append(golden)

print("\n" + "=" * 60)
dataset = EvaluationDataset(goldens=goldens_with_output)
print(f"\n✅ Created dataset with {len(dataset.goldens)} test cases")

## 📊 Part 6: Configure DeepEval Metrics

We use **3 metrics** that represent both categories:

| Metric | Type | Uses Context? | What It Measures |
|--------|------|---------------|------------------|
| `AnswerRelevancy` | 🆓 Referenceless | ❌ No | Is response relevant to the question? |
| `Faithfulness` | 📋 Reference-based | ✅ Yes | Is response grounded in tool outputs? |
| `Hallucination` | 📋 Reference-based | ✅ Yes | Does response make things up? |

> 💡 **Why these metrics?**  
> - `AnswerRelevancy`: Ensures agent addresses the user's question  
> - `Faithfulness`: Ensures agent uses tool outputs correctly  
> - `Hallucination`: Catches fabricated numbers or facts

In [ ]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric, HallucinationMetric
from deepeval.test_case import LLMTestCase

# 🧠 Judge model for evaluation
EVAL_MODEL = "gpt-4o-mini"

# 📊 Configure metrics
metrics = [
    # 🆓 REFERENCELESS: Only needs input + actual_output
    AnswerRelevancyMetric(threshold=0.7, model=EVAL_MODEL),
    
    # 📋 REFERENCE-BASED: Need retrieval_context (tool outputs)
    FaithfulnessMetric(threshold=0.7, model=EVAL_MODEL),
    HallucinationMetric(threshold=0.5, model=EVAL_MODEL),
]

print("📊 Metrics configured:\n")
print("  🆓 Referenceless (no ground truth needed):")
print(f"     • AnswerRelevancy (threshold: 0.7)")
print("\n  📋 Reference-based (uses retrieval_context):")
print(f"     • Faithfulness (threshold: 0.7)")
print(f"     • Hallucination (threshold: 0.5)")

In [ ]:
# 🔄 Convert Goldens to LLMTestCases
test_cases = [
    LLMTestCase(
        input=g.input,
        actual_output=g.actual_output,
        context=g.retrieval_context,           # 📋 For Hallucination metric
        retrieval_context=g.retrieval_context,  # 📋 For Faithfulness metric
    )
    for g in dataset.goldens
]

print(f"✅ Created {len(test_cases)} LLMTestCases")
print("\n💡 Note: retrieval_context = tool outputs = ground truth for reference-based metrics")

## 🔬 Part 7: Run DeepEval Evaluation

Now we run the LLM-as-Judge evaluation on all test cases.

In [ ]:
print("🔬 Running DeepEval evaluation...")
print("   This may take a few minutes as the judge LLM evaluates each test case.\n")

results = evaluate(
    test_cases=test_cases,
    metrics=metrics,
)

print("\n✅ Evaluation complete!")

## 📈 Part 8: Analyze Results

Let's examine the evaluation results in detail.

In [ ]:
import pandas as pd

# 📊 Build results dataframe
results_data = []

for tc, golden in zip(test_cases, dataset.goldens):
    meta = golden.additional_metadata
    row = {
        "test_id": meta["test_id"], 
        "category": meta["category"]
    }
    
    # Evaluate each metric
    for metric in metrics:
        metric.measure(tc)
        name = metric.__class__.__name__.replace("Metric", "")
        row[name] = round(metric.score, 2)
        row[f"{name}_pass"] = "✅" if metric.is_successful() else "❌"
    
    results_data.append(row)

df = pd.DataFrame(results_data)

print("📊 RESULTS BY TEST CASE\n")
print(df.to_string(index=False))

In [ ]:
# 📊 Summary by category
print("📊 AVERAGE SCORES BY CATEGORY\n")
score_cols = ["AnswerRelevancy", "Faithfulness", "Hallucination"]
summary = df.groupby("category")[score_cols].mean().round(2)
print(summary.to_string())

# 🎨 Visual representation
print("\n" + "=" * 50)
print("📈 CATEGORY PERFORMANCE")
print("=" * 50)
for cat in summary.index:
    scores = summary.loc[cat]
    avg = scores.mean()
    bar = "█" * int(avg * 20) + "░" * (20 - int(avg * 20))
    print(f"\n{cat}")
    print(f"  {bar} {avg:.0%}")

In [ ]:
# 📊 Overall pass rates
print("📊 OVERALL PASS RATES\n")

pass_data = {
    "🆓 AnswerRelevancy": (df["AnswerRelevancy_pass"] == "✅").mean(),
    "📋 Faithfulness": (df["Faithfulness_pass"] == "✅").mean(),
    "📋 Hallucination": (df["Hallucination_pass"] == "✅").mean(),
}

for metric, rate in pass_data.items():
    bar = "█" * int(rate * 20) + "░" * (20 - int(rate * 20))
    status = "✅" if rate >= 0.8 else "⚠️" if rate >= 0.6 else "❌"
    print(f"  {metric}: {bar} {rate:.0%} {status}")

# Overall
all_pass = (
    (df["AnswerRelevancy_pass"] == "✅") & 
    (df["Faithfulness_pass"] == "✅") & 
    (df["Hallucination_pass"] == "✅")
).mean()
print(f"\n  🎯 All metrics pass: {all_pass:.0%}")

## 🔧 Part 9: Tool Call Validation

Beyond LLM-as-Judge metrics, we also validate that the agent used the correct tools.

In [ ]:
print("🔧 TOOL CALL VALIDATION\n")

tool_results = []
for golden in dataset.goldens:
    meta = golden.additional_metadata
    expected = set(meta["expected_tools"])
    actual = set(meta["actual_tools"])
    match = expected.issubset(actual)
    
    status = "✅" if match else "❌"
    print(f"  {status} {meta['test_id']}")
    if not match:
        print(f"      Expected: {list(expected)}")
        print(f"      Got: {list(actual)}")
    
    tool_results.append(match)

accuracy = sum(tool_results) / len(tool_results)
print(f"\n🎯 Tool Call Accuracy: {accuracy:.0%}")

## 🎓 Part 10: Key Takeaways

### 🔄 DeepEval Evaluation Flow

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│   Goldens   │ ──► │   Agent     │ ──► │ LLMTestCase │ ──► │  Metrics    │
│ (templates) │     │ Execution   │     │             │     │             │
└─────────────┘     └─────────────┘     └─────────────┘     └─────────────┘
                          │
                          ▼
                   Tool Outputs = retrieval_context
                   (ground truth for reference-based metrics)
```

---

### 📊 Metric Types Summary

| Type | Metrics | Required Parameters | Use Case |
|------|---------|---------------------|----------|
| 🆓 **Referenceless** | AnswerRelevancy, Toxicity, Bias | `input`, `actual_output` only | No labeled data needed |
| 📋 **Reference-based** | Faithfulness, Hallucination | + `retrieval_context` | Ground truth from tools |
| 📋 **Reference-based** | ContextualRecall, Precision | + `expected_output` | Need expected answers |

---

### ✅ Best Practices

1. **🔧 Tool outputs → retrieval_context**  
   Use tool results as ground truth for Faithfulness/Hallucination

2. **📁 Separate tools module**  
   Keep notebook focused on evaluation logic

3. **🏷️ Category-based Goldens**  
   Organize tests by loan type/scenario for better analysis

4. **🔍 Multi-level validation**  
   Combine LLM-as-Judge metrics with deterministic tool validation

5. **📈 Track by category**  
   Identify which scenarios need improvement

---

### 📚 Resources

- [DeepEval Documentation](https://docs.confident-ai.com/)
- [LangChain](https://python.langchain.com/)
- [LangGraph](https://langchain-ai.github.io/langgraph/)
- [DeepEval Metrics Guide](https://deepeval.com/docs/metrics-introduction)

---

**👍 Found this helpful? Please upvote!**